# N-body system

Consider a system of $N$ bodies under gravitational attraction. The masses are $(m_i)_{i\in I}$ and the gravitational constant is ${\mathcal G}$. The problem is to study and represent the motion of this system.

We use standard (Euclidian) coordinates $(x_i,y_i)_{i\in I}$ in some arbitrary inertial frame of reference.

The equations of the dynamics are given by:

\begin{equation*}
\left\{
\begin{array}{rcl}
\ddot{x_i} & = &
\sum_{j\not=i}{\mathcal G}m_j((x_j-x_i)^2+(y_j-y_i)^2)^{-\frac{3}{2}}(x_j-x_i)
\\
\ddot{y_i} & = &
\sum_{j\not=i}{\mathcal G}m_j((x_j-x_i)^2+(y_j-y_i)^2)^{-\frac{3}{2}}(y_j-y_i)
\end{array}
\right.
\end{equation*}

Documentation: [here](../doc/_build/html/odesimu.html)

In [2]:
%pylab
# for inline animation (but slower and more resource consuming):
#%pylab nbagg

import logging
from functools import partial
from collections import ChainMap
from ipyshow.odesimu.system import System
from ipyshow.odesimu.util import logger_hook
from ipyshow.util import set_defaults, set_helper, Helper

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [3]:
#----------------------------------------------------------------------------------------------------
@set_helper(
    'ML: list of masses of the bodies [kg]',
    'G: gravitational constant [m^3.kg^-1.sec^-2]',
)
class GNbody (System):
#----------------------------------------------------------------------------------------------------

    def __init__(self,*ML,G=None):
        self.ML,self.G = ML,G
        N = len(ML)
        MM = array(ML)*G
        ID = eye(N,dtype=bool)
        slices = slice(None,N),slice(N,2*N)
        slicesd = slice(2*N,3*N),slice(3*N,None)
        self.shadowshape = (N,2)
        def main(t,state,a=zeros((2,N,N),dtype=float),b=zeros((1,N,N),dtype=float)):
            r = state.copy()
            for s,v,sd in zip(slices,a,slicesd):
                z = state[s]
                v[...] = z[newaxis,:]-z[:,newaxis]
                r[s] = state[sd]
            b = sum(square(a),axis=0,keepdims=True)
            b *= sqrt(b)
            b[0][ID] = 1.
            a /= b
            for s,v in zip(slicesd,a): dot(v,MM,r[s])
            return r
        self.main = main
        def fordisplay(state):
            x,y = (state[s] for s in slices)
            live = tuple(zip(x,y))
            return live, live
        self.fordisplay = fordisplay

    def display(self,ax,refsize=50.,**ka):
        diag_s = ax.scatter((),(),s=refsize,marker='o',c='b')
        tails_l = tuple(ax.plot((),(),'y')[0] for m in self.ML)
        ax.set_title('trajectory:?')
        def disp(t,live,tail):
            diag_s.set_offsets(live)
            for t_l,xy in zip(tails_l,swapaxes(tail,0,1)): t_l.set_data(xy[:,0],xy[:,1])
            ax.relim()
            ax.autoscale_view()
        return super().display(ax,disp,**ka)

    @set_helper(
        'xydxdyL: list of quadruple with xy-position [m] and speed [m.sec^-1]',
    )
    def makestate(self,*xydxdyL):
        assert len(xydxdyL)==len(self.ML)
        for u in xydxdyL: assert len(u)==4
        return concatenate(list(zip(*xydxdyL)))

    @set_helper(
        'refsize: average size (area) of the bodies for display [pt^2]',
        *System.launch.helper
    )
    @set_defaults(taild=10,hooks=(partial(logger_hook,logger=logging.getLogger()),),**System.launch.defaults)
    def launch(self,*a,**ka): return super().launch(*a,**ka)

In [5]:
Helper(GNbody,GNbody.launch,GNbody.makestate)

**** <class '__main__.GNbody'> ****
    ML        :  list of masses of the bodies [kg]
    G         :  gravitational constant [m^3.kg^-1.sec^-2]
**** <function GNbody.launch at 0x1114d5ae8> ****
    refsize   :  average size (area) of the bodies for display [pt^2]
    maxtime   (inf       ):  total simulation time length [sec]
    srate     (25.0      ):  sampling rate [sec^-1]
    taild     (10        ):  shadow duration [sec]
    hooks     (...       ):  tuple of display hooks
**** <function GNbody.makestate at 0x1114d5510> ****
    xydxdyL   :  list of quadruple with xy-position [m] and speed [m.sec^-1]

In [6]:
logging.basicConfig(level='WARN') # can be changed dynamically by logger_hook
syst = GNbody(10,10,10,10,G=1)
#v = 1; u = 4; a = -pi/12; d = 0.
v = 1; u = 4; a = -pi/12; d = .005
syst.launch(ini=syst.makestate(
    (-u,-u,v*cos(a),v*sin(a)),
    (u,-u,-v*sin(a),v*cos(a)),
    (u,u,-v*cos(a),-v*sin(a)),
    (-u,u,v*sin(a),d-v*cos(a)),
    )
)